In [1]:
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import basedosdados as bd
import numpy as np


In [2]:

os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\CNES")

In [3]:
df= pd.read_csv('CNES_profissionais_saude_ocupacao.csv')
df

,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos
0,2008,"Agentes, assistentes e auxiliares administrativos",AP,1600303.0,35
1,2008,Técnicos de laboratórios de saúde e bancos de ...,AP,1600303.0,177
2,2008,Enfermeiros e afins,AP,1600303.0,159
3,2008,Cirurgiões-dentistas,DF,5300108.0,126
4,2008,Médicos,DF,NaN,699
...,...,...,...,...,...
1377631,2021,Agentes fiscais metrológicos e de qualidade,TO,1705102.0,1
1377632,2021,Assistentes sociais e economistas domésticos,TO,1714880.0,2
1377633,2021,Recepcionistas,TO,1706100.0,5
1377634,2021,Trabalhadores em serviços de promoção e apoio ...,TO,1717909.0,30


In [9]:
df['tipo_2'] = np.where(df['tipo_ocupacao'].str.contains('Psicólogos e psicanalistas'),'Psicólogos e psicanalistas',df['tipo_ocupacao'])

In [11]:
df = df[(df['tipo_2']=='Psicólogos e psicanalistas') & (df['ano']>2017)]

In [13]:
## Chamando a quantidade de população.  

pop = pd.read_csv("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\IBGE\\Censo_previa_pop.csv", decimal=',', sep=';')

In [14]:
df.head(20)

,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos,tipo_2
286509,2018,Psicólogos e psicanalistas,BA,2924009.0,11,Psicólogos e psicanalistas
286518,2018,Psicólogos e psicanalistas,GO,5208707.0,387,Psicólogos e psicanalistas
286520,2018,Psicólogos e psicanalistas,MG,3106200.0,605,Psicólogos e psicanalistas
286533,2018,Psicólogos e psicanalistas,PB,2507507.0,378,Psicólogos e psicanalistas
286556,2018,Psicólogos e psicanalistas,RS,4314902.0,293,Psicólogos e psicanalistas
286571,2018,Psicólogos e psicanalistas,SE,2800308.0,131,Psicólogos e psicanalistas
286590,2018,Psicólogos e psicanalistas,SP,3548906.0,29,Psicólogos e psicanalistas
286632,2018,Psicólogos e psicanalistas,AC,1200401.0,56,Psicólogos e psicanalistas
286644,2018,Psicólogos e psicanalistas,AL,2706307.0,16,Psicólogos e psicanalistas
286696,2018,Psicólogos e psicanalistas,AM,1303700.0,1,Psicólogos e psicanalistas


In [15]:
df['id_municipio'] = pd.to_numeric(df['id_municipio'])

In [16]:
x=df.pivot_table(index=['ano','sigla_uf','id_municipio','tipo_2'], values='quantidade_vinculos',aggfunc=np.sum)
y = pd.DataFrame(x)
df = y.reset_index()

In [ ]:
x=df.pivot_table(index=['ano','sigla_uf','id_municipio','tipo_2'], values='quantidade_vinculos',aggfunc=np.sum)
y = pd.DataFrame(x)
df = y.reset_index()

In [17]:
df1= df.merge(pop[['UF','NOME DO MUNICÍPIO','id_municipio','POPULAÇÃO']], left_on='id_municipio',right_on='id_municipio').sort_values('quantidade_vinculos', ascending=False)

In [18]:
df1['taxa'] = df1['quantidade_vinculos']/df1['POPULAÇÃO']*1000

In [19]:
sum(df1[df1['ano']==2021]['POPULAÇÃO'])

204288187.0

In [20]:
sum(df1[df1['ano']==2021]['quantidade_vinculos'])/sum(df1[df1['ano']==2021]['POPULAÇÃO'])*1000

0.19362353046874903

In [21]:
## validando alguns dados
df1[(df1['ano']==2021) & (df1['UF']=='RJ')].sort_values('taxa', ascending=False).head(30)

,ano,sigla_uf,id_municipio,tipo_2,quantidade_vinculos,UF,NOME DO MUNICÍPIO,POPULAÇÃO,taxa
12794,2021,RJ,3305307.0,Psicólogos e psicanalistas,12,RJ,São Sebastião do Alto,7707.0,1.557026
12810,2021,RJ,3305604.0,Psicólogos e psicanalistas,29,RJ,Silva Jardim,21509.0,1.348273
12538,2021,RJ,3300936.0,Psicólogos e psicanalistas,15,RJ,Carapebus,13360.0,1.122754
12718,2021,RJ,3304128.0,Psicólogos e psicanalistas,13,RJ,Quatis,12646.0,1.027993
12682,2021,RJ,3303609.0,Psicólogos e psicanalistas,39,RJ,Paracambi,41146.0,0.947844
12774,2021,RJ,3305000.0,Psicólogos e psicanalistas,32,RJ,São João da Barra,36518.0,0.876280
12542,2021,RJ,3300951.0,Psicólogos e psicanalistas,7,RJ,Comendador Levy Gasparian,8769.0,0.798267
12714,2021,RJ,3304110.0,Psicólogos e psicanalistas,16,RJ,Porto Real,20375.0,0.785276
12726,2021,RJ,3304151.0,Psicólogos e psicanalistas,17,RJ,Quissamã,21749.0,0.781645
12706,2021,RJ,3304003.0,Psicólogos e psicanalistas,20,RJ,Piraí,27996.0,0.714388


In [22]:
df1.columns

Index(['ano', 'sigla_uf', 'id_municipio', 'tipo_2', 'quantidade_vinculos',
       'UF', 'NOME DO MUNICÍPIO', 'POPULAÇÃO', 'taxa'],
      dtype='object')

In [23]:
## Renomear colunas

df = df1[['ano', 'sigla_uf', 'id_municipio', 'tipo_2', 'quantidade_vinculos',
        'NOME DO MUNICÍPIO', 'POPULAÇÃO', 'taxa']].rename(columns={'tipo_2':'tipo_ocupacao','UF':'sigla_uf', 'NOME DO MUNICÍPIO':'nome_municipio','POPULAÇÃO':'populacao_domiciliada' })

In [24]:
df

,ano,sigla_uf,id_municipio,tipo_ocupacao,quantidade_vinculos,nome_municipio,populacao_domiciliada,taxa
18968,2018,SP,3550308.0,Psicólogos e psicanalistas,2169,São Paulo,12200180.0,0.177784
18971,2021,SP,3550308.0,Psicólogos e psicanalistas,1897,São Paulo,12200180.0,0.155490
18970,2020,SP,3550308.0,Psicólogos e psicanalistas,1773,São Paulo,12200180.0,0.145326
18969,2019,SP,3550308.0,Psicólogos e psicanalistas,1746,São Paulo,12200180.0,0.143113
12747,2018,RJ,3304557.0,Psicólogos e psicanalistas,1396,Rio de Janeiro,6625849.0,0.210690
...,...,...,...,...,...,...,...,...
10631,2019,PI,2202251.0,Psicólogos e psicanalistas,1,Canavieira,3409.0,0.293341
10632,2020,PI,2202251.0,Psicólogos e psicanalistas,1,Canavieira,3409.0,0.293341
10633,2021,PI,2202251.0,Psicólogos e psicanalistas,1,Canavieira,3409.0,0.293341
10642,2018,PI,2202505.0,Psicólogos e psicanalistas,1,Caracol,10326.0,0.096843


In [25]:
df= df[['ano', 'sigla_uf', 'nome_municipio','id_municipio','populacao_domiciliada', 'tipo_ocupacao',
       'quantidade_vinculos',  
       'taxa']]

In [26]:
df

,ano,sigla_uf,nome_municipio,id_municipio,populacao_domiciliada,tipo_ocupacao,quantidade_vinculos,taxa
18968,2018,SP,São Paulo,3550308.0,12200180.0,Psicólogos e psicanalistas,2169,0.177784
18971,2021,SP,São Paulo,3550308.0,12200180.0,Psicólogos e psicanalistas,1897,0.155490
18970,2020,SP,São Paulo,3550308.0,12200180.0,Psicólogos e psicanalistas,1773,0.145326
18969,2019,SP,São Paulo,3550308.0,12200180.0,Psicólogos e psicanalistas,1746,0.143113
12747,2018,RJ,Rio de Janeiro,3304557.0,6625849.0,Psicólogos e psicanalistas,1396,0.210690
...,...,...,...,...,...,...,...,...
10631,2019,PI,Canavieira,2202251.0,3409.0,Psicólogos e psicanalistas,1,0.293341
10632,2020,PI,Canavieira,2202251.0,3409.0,Psicólogos e psicanalistas,1,0.293341
10633,2021,PI,Canavieira,2202251.0,3409.0,Psicólogos e psicanalistas,1,0.293341
10642,2018,PI,Caracol,2202505.0,10326.0,Psicólogos e psicanalistas,1,0.096843


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20473 entries, 18968 to 20472
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ano                    20473 non-null  int64  
 1   sigla_uf               20473 non-null  object 
 2   nome_municipio         20473 non-null  object 
 3   id_municipio           20473 non-null  float64
 4   populacao_domiciliada  20473 non-null  float64
 5   tipo_ocupacao          20473 non-null  object 
 6   quantidade_vinculos    20473 non-null  int64  
 7   taxa                   20473 non-null  float64
dtypes: float64(3), int64(2), object(3)
memory usage: 1.4+ MB


Subindo para o GBQ

In [28]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

In [30]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('nome_municipio','STRING',description='Nome do município da observação'),
 bigquery.SchemaField('id_municipio','FLOAT',description='Identificador do município pelo IBGE'),
 bigquery.SchemaField('populacao_domiciliada','FLOAT',description='População domiciliada (prévia do Censo 2022)'),
 bigquery.SchemaField('tipo_ocupacao','STRING',description='Qual a ocupação daquele vínculo'),
 bigquery.SchemaField('quantidade_vinculos','INTEGER',description='Quantidade de vinculos'),
 bigquery.SchemaField('taxa','FLOAT',description='Taxa de psicólogos e psicanalistas por 1000 habitantes')
 ]

In [31]:
table_ref = dataset_ref.table('CNES_psicologos_mil_habitantes') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=06b0ea2b-e084-491d-970f-fe2933dadae2>